In [1]:
import pandas as pd
import nltk
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from profanity_check import predict, predict_prob

In [2]:
import slangs
import string

<h2> Getting Abbreviation of words (Singkatan English) </h2>

In [3]:
from bs4 import BeautifulSoup
import urllib3
http = urllib3.PoolManager()

In [4]:
Abbr_dict={}
#Load daftar slang word dari api noslang.com
def getAbbr(alpha):
    global Abbr_dict
    r = http.request('GET','https://www.noslang.com/dictionary/'+alpha)
    soup = BeautifulSoup(r.data,'html.parser')
    
    for i in soup.findAll('div',{'class':'dictionary-word'}): 
        abbr = i.find('abbr')['title']
        Abbr_dict[i.find('span').text[:-2]] = abbr
    

In [5]:
linkDict=[]

for one in range(97,123):
    linkDict.append(chr(one))

In [6]:
for i in linkDict:
    getAbbr(i)

In [7]:
#Hapus punctuation
s = "pro."
table = str.maketrans(dict.fromkeys(string.punctuation))
new_s = s.translate(table) 

In [8]:
#Test abbreviations
Abbr_dict[new_s]

'professional'

<h2>Preprocess Raw Data</h2>

In [9]:
data = pd.read_csv("dota2_chat_final.csv", encoding="utf_8", usecols=["match", "slot", "text"])
data = data.astype({"match": int, "slot": int, "text": str})
data.head(10)

,match,slot,text
0,2,0,yes dog. yeah . fast and furious. too fas. hah...
1,2,2,no idiot. we too pro. lol
2,2,4,HAHAH. COMMEND ME TY. EZ
3,6,0,so ya mama likes dick ehh?. figures. ur not ev...
4,6,1,reprot. SAD. fucking reported axe. WORST HOOK ...
5,6,3,gg
6,6,4,axe is axe
7,6,8,sorry nex. i killed u . almost . gg
8,8,6,PUSH. not defending. dodger lc. swap commend t...
9,9,3,what. jeje fAM. free farming ls. not coming in...


<h3>Preprocessing Slang words and get the correct words</h3>

In [10]:
all_words = data.text.tolist()
all_words_no_punc = []
#Hapus punctuation pada semua kata di dataset
for i in range(len(all_words)):
    table = str.maketrans(dict.fromkeys(string.punctuation))
    all_words_no_punc.append(all_words[i].lower().translate(table))

In [11]:
all_words_no_punc_slangs = []
for j in range(len(all_words_no_punc)):
    #print(all_words_no_punc[j])
    line = all_words_no_punc[j].split()
    for i in line:
        #Pengecekan jika kata terdapat dalam daftar kata slangs, dan diganti dengan kata asli nya
        if i in slangs.CONTRACTION_MAP:
            line[line.index(i)] = slangs.CONTRACTION_MAP[i]
        #Pengecekan jika kata tidak terdapat dalam daftar kata slangs, cek langsung dengan abbreviation list
        elif i in Abbr_dict:
            line[line.index(i)] = Abbr_dict[i]
    all_words_no_punc_slangs.append(' '.join(line).lower())

In [12]:
print(all_words_no_punc_slangs[1])

no idiot we too professional laughing out loud


<h3>Bad and Insult Words Classification</h3>

In [13]:
#Bad or Insult words classification dari kata setelah preprocessing dengan profanity checker
for i in range(len(all_words_no_punc_slangs)):
    print(all_words_no_punc_slangs[i][0:30]+"...", predict_prob([all_words_no_punc_slangs[i]])*100)

yes dog yeah fast and furious ... [23.26983421]
no idiot we too professional l... [79.84248862]
hahah commend me thank you eas... [6.14697756]
so yeah mama likes penis ehh f... [54.48983115]
reprot sad fucking reported ax... [91.13929032]
good game... [9.53466983]
axe is axe... [7.8481115]
sorry nex i killed you almost ... [12.17227698]
push not defending dodger lc s... [3.6480961]
what jeje family free farming ... [99.87796668]
heartshaker man that silence o... [13.16431897]
yeah well played gege guys hah... [63.14388034]
wrong person fuck you thank yo... [80.06875283]
drow remember me last game hah... [18.52376047]
ec haha nca game commend me... [12.46278212]
extreme droll so good game 3 g... [3.30798525]
well played shit cunts... [92.63575019]
feed me go bot please don't be... [11.56084985]
happy no cour day i will sure ... [16.5493177]
480 ping okay again and again ... [5.01669912]
stupid medusa fucking medusa n... [97.49937818]
wind reproted alien here laugh... [8.06338138]
good g